# Testing Notebook

## 1. Some Theory about Recommender Systems

The main families of methods for RecSys are:

- Collaborative Filtering: This method makes automatic predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption of the collaborative filtering approach is that if a person A has the same opinion as a person B on a set of items, A is more likely to have B's opinion for a given item than that of a randomly chosen person.

- Content-Based Filtering: This method uses only information about the description and attributes of the items users has previously consumed to model user's preferences. In other words, these algorithms try to recommend items that are similar to those that a user liked in the past (or is examining in the present). In particular, various candidate items are compared with items previously rated by the user and the best-matching items are recommended.

- Hybrid methods: Recent research has demonstrated that a hybrid approach, combining collaborative filtering and content-based filtering could be more effective than pure approaches in some cases. These methods can also be used to overcome some of the common problems in recommender systems such as cold start and the sparsity problem.

https://www.kaggle.com/code/gspmoreira/recommender-systems-in-python-101

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

In [2]:
import scipy
import math
import random
import sklearn
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler

Example: https://www.kaggle.com/code/hendraherviawan/itembased-collaborative-filter-recommendation-r/report

## 2 Read Data from Parquets

In [3]:
sample = 0.01
MIN_TRANSACTIONS = 0

In [4]:
transactions = pd.read_parquet(f'data/transactions_train_sample{sample}_gt{MIN_TRANSACTIONS}transactions.parquet')
customers = pd.read_parquet(f'data/customers_train_sample{sample}_gt{MIN_TRANSACTIONS}transactions.parquet')
articles = pd.read_parquet(f'data/articles_train_sample{sample}_gt{MIN_TRANSACTIONS}transactions.parquet')

## 3. Join Dataframes

In [5]:
#### Join Transactions and Articles dataframes
transactions_articles_joined = transactions.join(articles.set_index('article_id'), on='article_id')

In [6]:
transactions_articles_joined.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week,product_code,prod_name,product_type_no,product_type_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,2018-09-20,49501769952275870,615508002,0.016932,1,0,615508,2086,265,1,...,4,0,0,1,0,15,0,1013,8,1937
1,2018-09-20,50555163119119323,496111007,0.033881,2,0,496111,283,252,2,...,1,0,0,1,0,6,10,1003,3,437
2,2018-09-20,50555163119119323,651273001,0.022017,2,0,651273,5778,252,2,...,0,0,0,1,0,6,10,1005,0,2303
3,2018-09-20,50555163119119323,613826004,0.042356,2,0,613826,18223,262,6,...,46,1,1,2,2,53,1,1007,9,6939
4,2018-09-20,120750557174833794,662888004,0.033881,1,0,662888,143,252,2,...,1,0,0,1,0,15,0,1003,3,1030


In [7]:
#### Join Transactions-Articles with Customers dataframes
features_joined = transactions_articles_joined.join(customers.set_index('customer_id'), on='customer_id')

In [8]:
features_joined.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week,product_code,prod_name,product_type_no,product_type_name,...,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,club_member_status,fashion_news_frequency,age,postal_code
0,2018-09-20,49501769952275870,615508002,0.016932,1,0,615508,2086,265,1,...,0,15,0,1013,8,1937,0,1,76.0,312383
1,2018-09-20,50555163119119323,496111007,0.033881,2,0,496111,283,252,2,...,0,6,10,1003,3,437,0,0,22.0,269457
2,2018-09-20,50555163119119323,651273001,0.022017,2,0,651273,5778,252,2,...,0,6,10,1005,0,2303,0,0,22.0,269457
3,2018-09-20,50555163119119323,613826004,0.042356,2,0,613826,18223,262,6,...,2,53,1,1007,9,6939,0,0,22.0,269457
4,2018-09-20,120750557174833794,662888004,0.033881,1,0,662888,143,252,2,...,0,15,0,1003,3,1030,0,1,30.0,32853


In [9]:
len(features_joined)

311438

## 4 Drop unused columns

Collaborative Filtering only needs customers_id and articles_id

In [10]:
features_prepared = features_joined[['customer_id', 'article_id']]

## 5 Collaborative filtering

Build a matrix of items for the client

### 5.1 Build customer-articles matrix

In [11]:
#Add column quantity
features_prepared['Quantity'] = 1

/var/folders/vw/p_x0ctqx1qggpkh4cs8z3st80000gn/T/ipykernel_7559/439893453.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_prepared['Quantity'] = 1


In [12]:
features_prepared.head()

,customer_id,article_id,Quantity
0,49501769952275870,615508002,1
1,50555163119119323,496111007,1
2,50555163119119323,651273001,1
3,50555163119119323,613826004,1
4,120750557174833794,662888004,1


In [13]:
customer_item_matrix = features_prepared.pivot_table(
    index='customer_id', 
    columns='article_id', 
    values='Quantity',
    aggfunc='sum'
)

In [14]:
customer_item_matrix

article_id,108775015,108775044,108775051,110065001,110065002,110065011,111565001,111565003,111586001,111593001,...,946282001,946387001,946748003,946748004,946764003,947060001,947168001,947509001,949551001,949551002
customer_id,,,,,,,,,,,,,,,,,,,,,
345001598676045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1845857727772358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1951136007097426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2100196506917948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2163812782565753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18441525297103009017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18441975589952711961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18442032299905897310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 We now have a matrix where each row represents the total quantities purchased for each product for each customer.

let's code 0-1 this data, so that a value of 1 means that the given product was bought by the given customer, and a value of 0 means that the given product was never bought by the given customer. Take a look at the following code:

In [15]:
# customer_item_matrix = customer_item_matrix.applymap(lambda x: 1 if x > 0 else 0)
customer_item_matrix = (customer_item_matrix>0).astype(np.int8) #.astype isn't the best approach, but much faster

In [16]:
customer_item_matrix

article_id,108775015,108775044,108775051,110065001,110065002,110065011,111565001,111565003,111586001,111593001,...,946282001,946387001,946748003,946748004,946764003,947060001,947168001,947509001,949551001,949551002
customer_id,,,,,,,,,,,,,,,,,,,,,
345001598676045,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1845857727772358,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1951136007097426,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2100196506917948,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2163812782565753,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18441525297103009017,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18441975589952711961,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18442032299905897310,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5.2 Build Customers Similarity Matrix

Calculate the cosine similarities between users

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
user_user_sim_matrix = pd.DataFrame(
    cosine_similarity(customer_item_matrix)
)

In [19]:
user_user_sim_matrix

,0,1,2,3,4,5,6,7,8,9,...,13552,13553,13554,13555,13556,13557,13558,13559,13560,13561
0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.086066,0.0
2,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13557,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0
13558,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0
13559,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0
13560,0.0,0.086066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0


In [20]:
user_user_sim_matrix.columns = customer_item_matrix.index

In [21]:
user_user_sim_matrix['customer_id'] = customer_item_matrix.index
user_user_sim_matrix = user_user_sim_matrix.set_index('customer_id')

In [22]:
user_user_sim_matrix

customer_id,345001598676045,1845857727772358,1951136007097426,2100196506917948,2163812782565753,4125270926874550,6435244965610522,6772158986428566,7252995555838753,7960544439277532,...,18435362578049473419,18436849338957934485,18437423707072802046,18439338335320450190,18440481852248141449,18441525297103009017,18441975589952711961,18442032299905897310,18446522881501826039,18446588669559240504
customer_id,,,,,,,,,,,,,,,,,,,,,
345001598676045,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1845857727772358,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.086066,0.0
1951136007097426,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2100196506917948,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2163812782565753,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18441525297103009017,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0
18441975589952711961,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0
18442032299905897310,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0


These are the 10 most similar clients to the b14bfba3ae0da5af6e9711059773acf713cd7bb9a2c940ddf570affb715988a0 client. Let's choose client 26f41c2913090e7a620df05975a52c604caf017c3110bb62596fdebd1aae4ba9 and discuss how we can recommend products using these results.

Lets identify both users:
- user_A: b14bfba3ae0da5af6e9711059773acf713cd7bb9a2c940ddf570affb715988a0
- user_B: 26f41c2913090e7a620df05975a52c604caf017c3110bb62596fdebd1aae4ba9

The strategy is as follows.

- First, we need to identify the items that user_A and user2 customers have already purchased.
- Then, let's find the products that target client user_B has not purchased, but client user_A has.
- Since these two customers have bought similar items in the past, we will assume that target customer user_B has a high probability of buying the items he or she has not bought, but customer user_A has.
- Finally, we are going to use this list of items and recommend them to target customer user_B.

### 5.3 Build the recommendations for each customer

In [23]:
# Create empty dataframe that will store the customers and their recommended products
column_names = ["customer_id", "recom_articles"]
customer_rec_articles_df = pd.DataFrame(columns = column_names)

In [24]:
%%capture --no-display 

for i in range(len(customer_item_matrix)-1):   
    customer_i_id = customer_item_matrix.index[i]
    items_to_recommend_to_i = set()

    items_bought_by_i = set(customer_item_matrix.loc[customer_i_id].iloc[customer_item_matrix.loc[customer_i_id].to_numpy().nonzero()].index)
    
    #Get most similar users to current user i
    similar_users = user_user_sim_matrix.iloc[i].sort_values(ascending=False)[0:10] #get just the 10 most similar clients
    similar_users = similar_users[similar_users!=0].to_frame() #Get only clients with similarity score greater than 0

    #Get articles bought by the j similar users
    for j in similar_users.index.values:
        customer_j_id = j
        items_bought_by_j = set(customer_item_matrix.loc[customer_j_id].iloc[customer_item_matrix.loc[customer_j_id].to_numpy().nonzero()].index)
        items_to_recommend = items_bought_by_j - items_bought_by_i
        #if(len(items_to_recommend)>0): items_to_recommend_to_i.append(items_to_recommend)
        items_to_recommend_to_i = set.union(items_to_recommend_to_i, items_to_recommend)

    customer_rec_articles_df = customer_rec_articles_df.append({'customer_id': customer_i_id, 'recom_articles':items_to_recommend_to_i}, ignore_index=True)


In [25]:
#Convert set of recommended articles to list
customer_rec_articles_df['recom_articles'] = customer_rec_articles_df['recom_articles'].apply(lambda x: list(x))

In [26]:
len(customer_rec_articles_df)

13561

In [27]:
customer_rec_articles_df.head()

,customer_id,recom_articles
0,345001598676045,"[875316001, 850176002, 778064003, 841383002, 6..."
1,1845857727772358,"[790096001, 790096002, 605106006, 715303001, 7..."
2,1951136007097426,"[300024064, 737040001, 824752001, 706016003, 6..."
3,2100196506917948,"[720635008, 355072001, 680262018, 864928001, 7..."
4,2163812782565753,"[835136002, 708352003, 794389001, 824085001, 7..."


Export dataframe to csv:

In [28]:
customer_rec_articles_df.to_csv(f'customers_recommended_articles_sample{sample}.csv', index=False)

## 6 FUTURE: Content-Based Filtering

https://www.kaggle.com/code/fabiendaniel/film-recommendation-engine/notebook

SIMILARITY

Criteria to determine if two products are similar:
1. Section and Type (both together)
2. Graphical appearence
3. Colour